# Basic usage of LRP
This example will show you best practices for using LRP,
building on the basics shown in the *Getting started* section.

We start out by loading a small convolutional neural network:

In [1]:
using ExplainableAI
using Flux

model = Chain(
    Chain(
        Conv((3, 3), 3 => 8, relu; pad=1),
        Conv((3, 3), 8 => 8, relu; pad=1),
        MaxPool((2, 2)),
        Conv((3, 3), 8 => 16; pad=1),
        BatchNorm(16, relu),
        Conv((3, 3), 16 => 8, relu; pad=1),
        BatchNorm(8, relu),
    ),
    Chain(
        Flux.flatten,
        Dense(2048 => 512, relu),
        Dropout(0.5),
        Dense(512 => 100, softmax)
    ),
);

This model contains two chains: the convolutional layers and the fully connected layers.

## Model preparation
### Stripping the output softmax
When using LRP, it is recommended to explain output logits instead of probabilities.
This can be done by stripping the output softmax activation from the model
using the `strip_softmax` function:

In [2]:
model = strip_softmax(model)

Chain(
  Chain(
    Conv((3, 3), 3 => 8, relu, pad=1),  # 224 parameters
    Conv((3, 3), 8 => 8, relu, pad=1),  # 584 parameters
    MaxPool((2, 2)),
    Conv((3, 3), 8 => 16, pad=1),       # 1_168 parameters
    BatchNorm(16, relu),                # 32 parameters, plus 32
    Conv((3, 3), 16 => 8, relu, pad=1),  # 1_160 parameters
    BatchNorm(8, relu),                 # 16 parameters, plus 16
  ),
  Chain(
    Flux.flatten,
    Dense(2048 => 512, relu),           # 1_049_088 parameters
    Dropout(0.5),
    Dense(512 => 100),                  # 51_300 parameters
  ),
)         # Total: 16 trainable arrays, 1_103_572 parameters,
          # plus 4 non-trainable, 48 parameters, summarysize 4.213 MiB.

If you don't remove the output softmax,
model checks will fail.

### Canonizing the model
LRP is not invariant to a model's implementation.
Applying the `GammaRule` to two linear layers in a row will yield different results
than first fusing the two layers into one linear layer and then applying the rule.
This fusing is called "canonization" and can be done using the `canonize` function:

In [3]:
model = canonize(model)

Chain(
  Chain(
    Conv((3, 3), 3 => 8, relu, pad=1),  # 224 parameters
    Conv((3, 3), 8 => 8, relu, pad=1),  # 584 parameters
    MaxPool((2, 2)),
    Conv((3, 3), 8 => 16, pad=1),       # 1_168 parameters
    BatchNorm(16, relu),                # 32 parameters, plus 32
    Conv((3, 3), 16 => 8, relu, pad=1),  # 1_160 parameters
    BatchNorm(8, relu),                 # 16 parameters, plus 16
  ),
  Chain(
    Flux.flatten,
    Dense(2048 => 512, relu),           # 1_049_088 parameters
    Dropout(0.5),
    Dense(512 => 100),                  # 51_300 parameters
  ),
)         # Total: 16 trainable arrays, 1_103_572 parameters,
          # plus 4 non-trainable, 48 parameters, summarysize 4.213 MiB.

### Flattening the model
ExplainableAI.jl's LRP implementation supports nested Flux Chains and Parallel layers.
However, it is recommended to flatten the model before analyzing it.

LRP is implemented by first running a forward pass through the model,
keeping track of the intermediate activations, followed by a backward pass
that computes the relevances.

To keep the LRP implementation simple and maintainable,
ExplainableAI.jl does not pre-compute "nested" activations.
Instead, for every internal chain, a new forward pass is run to compute activations.

By "flattening" a model, this overhead can be avoided.
For this purpose, ExplainableAI.jl provides the function `flatten_model`:

In [4]:
model_flat = flatten_model(model)

Chain(
  Conv((3, 3), 3 => 8, relu, pad=1),    # 224 parameters
  Conv((3, 3), 8 => 8, relu, pad=1),    # 584 parameters
  MaxPool((2, 2)),
  Conv((3, 3), 8 => 16, pad=1),         # 1_168 parameters
  BatchNorm(16, relu),                  # 32 parameters, plus 32
  Conv((3, 3), 16 => 8, relu, pad=1),   # 1_160 parameters
  BatchNorm(8, relu),                   # 16 parameters, plus 16
  Flux.flatten,
  Dense(2048 => 512, relu),             # 1_049_088 parameters
  Dropout(0.5),
  Dense(512 => 100),                    # 51_300 parameters
)         # Total: 16 trainable arrays, 1_103_572 parameters,
          # plus 4 non-trainable, 48 parameters, summarysize 4.212 MiB.

This function is called by default when creating an LRP analyzer.
Note that we pass the unflattened model to the analyzer, but `analyzer.model` is flattened:

In [5]:
analyzer = LRP(model)
analyzer.model

Chain(
  Conv((3, 3), 3 => 8, relu, pad=1),    # 224 parameters
  Conv((3, 3), 8 => 8, relu, pad=1),    # 584 parameters
  MaxPool((2, 2)),
  Conv((3, 3), 8 => 16, pad=1),         # 1_168 parameters
  BatchNorm(16, relu),                  # 32 parameters, plus 32
  Conv((3, 3), 16 => 8, relu, pad=1),   # 1_160 parameters
  BatchNorm(8, relu),                   # 16 parameters, plus 16
  Flux.flatten,
  Dense(2048 => 512, relu),             # 1_049_088 parameters
  Dropout(0.5),
  Dense(512 => 100),                    # 51_300 parameters
)         # Total: 16 trainable arrays, 1_103_572 parameters,
          # plus 4 non-trainable, 48 parameters, summarysize 4.212 MiB.

If this flattening is not desired, it can be disabled
by passing the keyword argument `flatten=false` to the `LRP` constructor.

## LRP rules
By default, the `LRP` constructor will assign the `ZeroRule` to all layers.

In [6]:
LRP(model)

LRP(
  Conv((3, 3), 3 => 8, relu, pad=1)  => ZeroRule(),
  Conv((3, 3), 8 => 8, relu, pad=1)  => ZeroRule(),
  MaxPool((2, 2))                    => ZeroRule(),
  Conv((3, 3), 8 => 16, pad=1)       => ZeroRule(),
  BatchNorm(16, relu)                => ZeroRule(),
  Conv((3, 3), 16 => 8, relu, pad=1) => ZeroRule(),
  BatchNorm(8, relu)                 => ZeroRule(),
  Flux.flatten                       => ZeroRule(),
  Dense(2048 => 512, relu)           => ZeroRule(),
  Dropout(0.5)                       => ZeroRule(),
  Dense(512 => 100)                  => ZeroRule(),
)

This analyzer will return heatmaps that look identical to `InputTimesGradient`.

LRP's strength lies in assigning different rules to different layers,
based on their functionality in the neural network[^1].
ExplainableAI.jl implements many LRP rules out of the box,
but it is also possible to *implement custom rules*.

To assign different rules to different layers,
use one of the composites presets,
or create your own composite, as described in
*Assigning rules to layers*.

In [7]:
composite = EpsilonPlusFlat() # using composite preset EpsilonPlusFlat

Composite(
  GlobalTypeMap(  # all layers
    Flux.Conv               => ZPlusRule(),
    Flux.ConvTranspose      => ZPlusRule(),
    Flux.CrossCor           => ZPlusRule(),
    Flux.Dense              => EpsilonRule{Float32}(1.0f-6),
    typeof(NNlib.dropout)   => PassRule(),
    Flux.AlphaDropout       => PassRule(),
    Flux.Dropout            => PassRule(),
    Flux.BatchNorm          => PassRule(),
    typeof(Flux.flatten)    => PassRule(),
    typeof(MLUtils.flatten) => PassRule(),
    typeof(identity)        => PassRule(),
 ),
  FirstLayerTypeMap(  # first layer
    Flux.Conv          => FlatRule(),
    Flux.ConvTranspose => FlatRule(),
    Flux.CrossCor      => FlatRule(),
    Flux.Dense         => FlatRule(),
 ),
)

In [8]:
LRP(model, composite)

LRP(
  Conv((3, 3), 3 => 8, relu, pad=1)  => FlatRule(),
  Conv((3, 3), 8 => 8, relu, pad=1)  => ZPlusRule(),
  MaxPool((2, 2))                    => ZeroRule(),
  Conv((3, 3), 8 => 16, pad=1)       => ZPlusRule(),
  BatchNorm(16, relu)                => PassRule(),
  Conv((3, 3), 16 => 8, relu, pad=1) => ZPlusRule(),
  BatchNorm(8, relu)                 => PassRule(),
  Flux.flatten                       => PassRule(),
  Dense(2048 => 512, relu)           => EpsilonRule{Float32}(1.0f-6),
  Dropout(0.5)                       => PassRule(),
  Dense(512 => 100)                  => EpsilonRule{Float32}(1.0f-6),
)

## Computing layerwise relevances
If you are interested in computing layerwise relevances,
call `analyze` with an LRP analyzer and the keyword argument
`layerwise_relevances=true`.

The layerwise relevances can be accessed in the `extras` field
of the returned `Explanation`:

In [9]:
input = rand(Float32, 32, 32, 3, 1) # dummy input for our convolutional neural network

expl = analyze(input, analyzer; layerwise_relevances=true)
expl.extras.layerwise_relevances

12-element Vector{Array{Float32}}:
 [-0.07580648 -0.0025950721 … 0.97768915 -0.2860697; -0.09138524 -0.043458026 … 0.321484 0.69353867; … ; -0.7507186 -1.091334 … -0.3270273 0.009739506; -0.04051595 -0.21606395 … 0.083900295 0.1819309;;; 0.07556589 0.0069672223 … -0.0015038247 -0.5452235; -0.009782139 0.019828854 … 0.7607348 -0.8543455; … ; -0.10683122 -0.033683266 … -0.13058482 -0.521727; 0.115392596 0.0826543 … -0.014022934 0.025658706;;; -0.26691276 0.10041063 … 0.8359216 0.4924462; 0.026187466 0.097162195 … -0.04315075 0.80675596; … ; -0.2479432 0.27331322 … 0.55016387 -0.00015570153; -0.018626925 0.03584828 … -0.13687198 0.12594768;;;;]
 [-0.04570253 -0.02841987 … 0.04485167 0.4404947; -0.0 0.14312291 … -0.11894592 0.40282387; … ; 0.0 0.2164924 … 0.040280335 -0.1175853; -0.0 0.02479314 … -0.27632552 -0.13530312;;; -0.06534273 -0.024968136 … 0.023927515 0.107557096; -0.17578638 -0.082794234 … 0.11516758 0.57107013; … ; 0.016135523 -0.17565645 … 0.21137218 0.15428472; -0.0 -0.002426

Note that the layerwise relevances are only kept for layers in the outermost `Chain` of the model.
When using our unflattened model, we only obtain three layerwise relevances,
one for each chain in the model and the output relevance:

In [10]:
analyzer = LRP(model; flatten=false) # use unflattened model

expl = analyze(input, analyzer; layerwise_relevances=true)
expl.extras.layerwise_relevances

3-element Vector{Array{Float32}}:
 [0.2750725 0.00064664165 … -0.28899392 0.03692248; -0.040362455 -0.00037220112 … -0.5054151 -0.5339077; … ; -0.37226355 -0.32259902 … 0.373658 -0.04368532; -0.010391147 -0.21833049 … -0.015856722 -0.19409744;;; -0.021079237 -0.019027624 … -0.05500172 -0.07021334; -0.008057456 -0.02212938 … 0.57944876 0.04072019; … ; 0.024396492 -0.014354983 … 0.031924866 0.24452856; 0.098963335 -0.49938625 … 0.06593048 -0.041693144;;; 0.15466395 0.15488096 … 0.106925406 -0.06634142; -0.0031339556 -0.08094452 … -0.18928334 0.029980607; … ; 0.07512347 0.023309221 … -0.45515084 -0.00071278494; -0.026204761 -0.12524994 … 0.07562646 -0.11528954;;;;]
 [0.0 0.0 … 0.0 0.0063316487; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0038527502 0.0 … 0.0010413068 -0.00023109045; … ; -0.0008897862 0.003034069 … 0.026898839 0.0; -0.00020266941 0.0024530864 … -0.0067600855 0.00014757732;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0

## Performance tips
### Using LRP without a GPU
Since ExplainableAI.jl's LRP implementation makes use of
[Tullio.jl](https://github.com/mcabbott/Tullio.jl),
analysis can be accelerated by loading either
- a package from the [JuliaGPU](https://juliagpu.org) ecosystem,
  e.g. [CUDA.jl](https://github.com/JuliaGPU/CUDA.jl), if a GPU is available
- [LoopVectorization.jl](https://github.com/JuliaSIMD/LoopVectorization.jl)
  if only a CPU is available.

This only requires loading the LoopVectorization.jl package before ExplainableAI.jl:
```julia
using LoopVectorization
using ExplainableAI
```

[^1]: G. Montavon et al., [Layer-Wise Relevance Propagation: An Overview](https://link.springer.com/chapter/10.1007/978-3-030-28954-6_10)

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*